In [159]:
#Блок в рамках которого импотряться математические подсчеты, падас и нампай
import pandas as pd
import numpy as np
from scipy.stats import iqr
from scipy import stats
from scipy.stats import skew
from scipy.stats import kurtosis
import scipy.stats
import scipy as sp
import scipy.stats as stats
from scipy.stats import t

#Блок в рамках которого импотряться графики
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns 
from matplotlib.pyplot import figure
sns.set_style('darkgrid')

#Блок в рамках которого импотряться статистические пакеты
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.regression.linear_model import OLS
from sklearn.decomposition import PCA
from statsmodels.stats.diagnostic import het_white, het_goldfeldquandt, het_breuschpagan
from statsmodels.stats.outliers_influence import reset_ramsey
from statsmodels.stats.diagnostic import compare_encompassing 
from linearmodels.iv import IV2SLS

#Блок в рамках которого импотряться операционные пакеты
import os


# Содержание работы
7. [Endogeneity](#subparagraph0)  - решение проблемы эндогенности

In [160]:
data = pd.read_csv('data\PhonesDF_years_corrected_v2.csv').set_index('phone_title')
all_spec = pd.read_csv('data\PhonesPriceInKenya_v2.csv').set_index('Phone Title')['Specs Score']
# data.shape

In [161]:
data = data.join(pd.DataFrame(all_spec))
data['Specs Score'] = data['Specs Score'].str.replace('%', '').astype(int)
data = add_constant(data)
data.shape

(559, 34)

### 7. Endogeneity

В рамках датасета можно отметить, что пропущены характеристики, отражающие производительность смартфона (графический и центральный процессоры). В то же время такая характеристика как ёмкость батареи ('battery_capacity') может иметь коореляцию с мощностью графического и/или центрального процессора. Следовательно, это может приводить к к проблеме эндогенности.

В рамках решения данной проблемы воспользуюмся 2SLS

#### 7.1 battery_capacity - эндогенный

In [162]:
mask_exog_and_instr3 = ['const', 'rating', 'disp_diag', 'camera', 'storage',
               'density', 'Other_oper_syst', 'battery_type_LiPo', 'brand_Apple',
               'brand_BlackBerry', 'brand_Cubot', 'brand_Gionee', 'brand_Google',
               'brand_HTC', 'brand_Honor', 'brand_Huawei', 'brand_Infinix', 'brand_LG',
               'brand_Lava', 'brand_Lenovo', 'brand_Motorola', 'brand_Nokia',
               'brand_OnePlus', 'brand_Oppo', 'brand_Realme', 'brand_Samsung',
               'brand_Sony', 'brand_Tecno', 'brand_Vivo', 'Specs Score', 'year_of_release', 'ram']

mask_exog3 = ['const', 'rating', 'disp_diag', 'camera', 'storage',
               'density', 'Other_oper_syst', 'battery_type_LiPo', 'brand_Apple',
               'brand_BlackBerry', 'brand_Cubot', 'brand_Gionee', 'brand_Google',
               'brand_HTC', 'brand_Honor', 'brand_Huawei', 'brand_Infinix', 'brand_LG',
               'brand_Lava', 'brand_Lenovo', 'brand_Motorola', 'brand_Nokia',
               'brand_OnePlus', 'brand_Oppo', 'brand_Realme', 'brand_Samsung',
               'brand_Sony', 'brand_Tecno', 'brand_Vivo', 'year_of_release', 'ram']

In [163]:
first_step_2SLS3 = IV2SLS(dependent = data['battery_capacity'],  exog = data[mask_exog_and_instr3], endog = None, instruments= None, weights=  None)
first_step_2SLS3 = first_step_2SLS3.fit()
first_step_2SLS3

OLS Estimation Summary                            
==============================================================================
Dep. Variable:       battery_capacity   R-squared:                      0.7142
Estimator:                        OLS   Adj. R-squared:                 0.6974
No. Observations:                 559   F-statistic:                    3654.6
Date:                Wed, May 10 2023   P-value (F-stat)                0.0000
Time:                        23:34:19   Distribution:                 chi2(31)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
=====================================================================================
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
const                -97.011     39.574    -2.4514     0.0142     -174.57     -19.448
rating               -0.0498     0.0284    -1.7552     0.0792     -0.1053      0.0058
disp_diag             0.8751     0.0662     13.214     0.0000      0.7453      1.0049
camera               -0.0091     0.0032    -2.8430     0.0045     -0.0154     -0.0028
storage              -0.0009     0.0003    -2.7639     0.0057     -0.0016     -0.0003
density              -0.0054     0.0089    -0.6090     0.5425     -0.0228      0.0120
Other_oper_syst      -0.0628     0.4289    -0.1463     0.8836     -0.9034      0.7778
battery_type_LiPo     0.1849     0.0551     3.3577     0.0008      0.0770      0.2928
brand_Apple          -0.4626     0.0958    -4.8307     0.0000     -0.6503     -0.2749
brand_BlackBerry      0.5873     0.1801     3.2612     0.0011      0.2343      0.9402
brand_Cubot           0.2150     0.3558     0.6041     0.5458     -0.4825      0.9124
brand_Gionee          0.1200     0.0742     1.6168     0.1059     -0.0255      0.2654
brand_Google         -0.2668     0.1474    -1.8102     0.0703     -0.5556      0.0221
brand_HTC            -0.2287     0.0930    -2.4585     0.0140     -0.4111     -0.0464
brand_Honor          -0.5683     0.0755    -7.5279     0.0000     -0.7163     -0.4204
brand_Huawei         -0.3153     0.0708    -4.4561     0.0000     -0.4539     -0.1766
brand_Infinix         0.0352     0.0869     0.4056     0.6850     -0.1351      0.2055
brand_LG             -0.3678     0.0943    -3.9007     0.0001     -0.5525     -0.1830
brand_Lava           -0.6178     0.2212    -2.7931     0.0052     -1.0513     -0.1843
brand_Lenovo         -0.4094     0.1259    -3.2519     0.0011     -0.6561     -0.1626
brand_Motorola        0.1927     0.3704     0.5202     0.6029     -0.5333      0.9187
brand_Nokia          -0.4696     0.1019    -4.6067     0.0000     -0.6694     -0.2698
brand_OnePlus        -0.4570     0.0866    -5.2801     0.0000     -0.6267     -0.2874
brand_Oppo           -0.1866     0.0828    -2.2533     0.0242     -0.3489     -0.0243
brand_Realme          0.3252     0.1345     2.4183     0.0156      0.0616      0.5888
brand_Samsung        -0.0712     0.0862    -0.8263     0.4086     -0.2402      0.0977
brand_Sony           -0.1556     0.1373    -1.1334     0.2570     -0.4247      0.1135
brand_Tecno           0.1768     0.1317     1.3427     0.1794     -0.0813      0.4349
brand_Vivo            0.6788     0.0710     9.5565     0.0000      0.5395      0.8180
Specs Score           0.0103     0.0025     4.0916     0.0000      0.0054      0.0153
year_of_release       0.0473     0.0197     2.3991     0.0164      0.0087      0.0859
ram                  -0.0333     0.0093    -3.6002     0.0003     -0.0515     -0.0152
=====================================================================================
OLSResults, id: 0x1cc8b326f70

In [165]:
res_iv3 = IV2SLS(data['log_price'], data[mask_exog3], data['battery_capacity'], 
                data['Specs Score']).fit()
print(res_iv3)


                          IV-2SLS Estimation Summary                          
Dep. Variable:              log_price   R-squared:                      0.5749
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5499
No. Observations:                 559   F-statistic:                    5020.9
Date:                Wed, May 10 2023   P-value (F-stat)                0.0000
Time:                        23:34:39   Distribution:                 chi2(31)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
const                -56.705     46.456    -1.2206     0.2222     -147.76      34.347
rating                0.

In [166]:
print(res_iv3.wu_hausman())


Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 0.0305
P-value: 0.8614
Distributed: F(1,526)


In [167]:
print(res_iv3.sargan)


Invalid test statistic
Test requires more instruments than endogenous variables.
Sargan's test of overidentification


Относительно тестов Хаусмана - гипотезы о эндогенности отвергаются, но это может связанно как с невалидностью инструмента, так с эндогенностью регрессора.